# 02-3 강아지와 고양이 사진 분류 모델의 성능 개선하기

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/02-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## ResNet 모델 만들기

### 배치 정규화

In [1]:
import keras
from keras import layers

inputs = layers.Input(shape=(224, 224, 3))
x = layers.ZeroPadding2D(padding=3)(inputs)
x = layers.Conv2D(64, 7, strides=2)(x)
x = layers.BatchNormalization(epsilon=1e-5)(x)
x = layers.Activation('relu')(x)

In [2]:
x = layers.ZeroPadding2D(padding=1)(x)
x = layers.MaxPooling2D(3, strides=2)(x)

### 잔차 스택 만들기

In [3]:
def build_stack(x):
    # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1을 사용합니다
    x = residual_stack(x, 3, 64, first_stride=1)
    # 두 번째~네 번째 잔차 블록을 만듭니다
    for blocks, filters in [(4, 128), (6, 256), (3, 512)]:
        x = residual_stack(x, blocks, filters, first_stride=2)
    return x

def residual_stack(x, blocks, filters, first_stride=2):
    # 첫 번째 잔차 블록은 합성곱 스킵 연결을 사용하고
    # 이 잔차 블록의 첫 번째 합성곱 스트라이드는 first_stride입니다.
    x = residual_block(x, filters, first_stride=first_stride, conv_skip=True)
    for _ in range(1, blocks):
        # 나머지 잔차 블록의 첫 번째 합성곱 스트라이드는 1입니다.
        x = residual_block(x, filters, first_stride=1, conv_skip=False)
    return x

In [4]:
def residual_block(x, filters, first_stride=1, conv_skip=False):
    skip_conn = x
    # 합성곱과 배치 정규화, 렐루 활성화 함수를 반복합니다
    # 1x1, filters개 필터, 스트라이드는 first_stride에 따라 1 또는 2
    x = layers.Conv2D(filters=filters, kernel_size=1,
                      strides=first_stride)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    # 3x3, filters개 필터
    x = layers.Conv2D(filters=filters, kernel_size=3,
                      padding='same')(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    # 1x1, filters*4개 필터
    x = layers.Conv2D(filters=filters*4, kernel_size=1)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    # conv_skip이 True이면 1x1 합성곱을 사용해 채널 크기를 filters*4로 늘립니다
    if conv_skip == True:
        skip_conn = layers.Conv2D(filters=filters*4, kernel_size=1,
                                  strides=first_stride)(skip_conn)
        skip_conn = layers.BatchNormalization(epsilon=1e-5)(skip_conn)
    x = layers.Add()([skip_conn, x])
    x = layers.Activation('relu')(x)
    return x

### ResNet 모델 만들기

In [5]:
x = build_stack(x)

In [6]:
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1000, activation='softmax')(x)

In [7]:
model = keras.Model(inputs, outputs)

In [8]:
def build_stack101(x):
    # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1을 사용합니다
    x = residual_stack(x, 3, 64, first_stride=1)
    # 두 번째~네 번째 잔차 블록을 만듭니다
    for blocks, filters in [(4, 128), (23, 256), (3, 512)]:
        x = residual_stack(x, blocks, filters, first_stride=2)
    return x

In [9]:
def build_stack152(x):
    # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1을 사용합니다
    x = residual_stack(x, 3, 64, first_stride=1)
    # 두 번째~네 번째 잔차 블록을 만듭니다
    for blocks, filters in [(8, 128), (36, 256), (3, 512)]:
        x = residual_stack(x, blocks, filters, first_stride=2)
    return x

## 강아지와 고양이 사진 분류하기

In [10]:
!gdown 1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
!unzip cat-dog-images.zip

Downloading...
From: https://drive.google.com/uc?id=1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
To: /content/cat-dog-images.zip
100% 182k/182k [00:00<00:00, 61.4MB/s]
Archive:  cat-dog-images.zip
   creating: images/
  inflating: images/dog.png          
  inflating: images/cat.png          


In [11]:
from PIL import Image
import numpy as np
from keras.applications import resnet

dog_png = Image.open('images/dog.png')
resnet_prep_dog = resnet.preprocess_input(np.array(dog_png))

In [12]:
resnet50 = keras.applications.ResNet50()
predictions = resnet50.predict(resnet_prep_dog[np.newaxis,:])

resnet.decode_predictions(predictions)

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[[('n02099712', 'Labrador_retriever', np.float32(0.38535273)),
  ('n02099601', 'golden_retriever', np.float32(0.089699574)),
  ('n02100735', 'English_setter', np.float32(0.04212418)),
  ('n02106166', 'Border_collie', np.float32(0.03777442)),
  ('n02101388', 'Brittany_spaniel', np.float32(0.030700428))]]

In [13]:
cat_png = Image.open('images/cat.png')
resnet_prep_cat = resnet.preprocess_input(np.array(cat_png))
predictions = resnet50.predict(resnet_prep_cat[np.newaxis,:])

resnet.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


[[('n02123045', 'tabby', np.float32(0.8686101)),
  ('n02124075', 'Egyptian_cat', np.float32(0.050774965)),
  ('n02123159', 'tiger_cat', np.float32(0.042567052)),
  ('n07930864', 'cup', np.float32(0.0027631458)),
  ('n03443371', 'goblet', np.float32(0.0020991666))]]

## 미니 프로젝트

In [14]:
from keras.utils import load_img
from keras.applications import inception_v3

dog_png = load_img("images/dog.png", target_size=(299, 299))
incep_prep_dog = inception_v3.preprocess_input(np.array(dog_png))

In [15]:
inception = keras.applications.InceptionV3()
predictions = inception.predict(incep_prep_dog[np.newaxis,:])
inception_v3.decode_predictions(predictions)

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


[[('n02104029', 'kuvasz', np.float32(0.13835053)),
  ('n02099712', 'Labrador_retriever', np.float32(0.07777295)),
  ('n02106166', 'Border_collie', np.float32(0.07198329)),
  ('n02111500', 'Great_Pyrenees', np.float32(0.06614903)),
  ('n02099601', 'golden_retriever', np.float32(0.02838334))]]

In [16]:
cat_png = load_img("images/cat.png", target_size=(299, 299))
incep_prep_cat = inception_v3.preprocess_input(np.array(cat_png))
predictions = inception.predict(incep_prep_cat[np.newaxis,:])
inception_v3.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


[[('n02124075', 'Egyptian_cat', np.float32(0.68673676)),
  ('n02123159', 'tiger_cat', np.float32(0.13263007)),
  ('n02123045', 'tabby', np.float32(0.04215029)),
  ('n04040759', 'radiator', np.float32(0.0016103369)),
  ('n02971356', 'carton', np.float32(0.0011297755))]]

In [17]:
from keras.applications import inception_resnet_v2 as incep_res_v2

incep_res_prep_dog = incep_res_v2.preprocess_input(np.array(dog_png))
inception_resnet = keras.applications.InceptionResNetV2()
predictions = inception_resnet.predict(incep_res_prep_dog[np.newaxis,:])
incep_res_v2.decode_predictions(predictions)

225209952/225209952 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


[[('n02099712', 'Labrador_retriever', np.float32(0.6563314)),
  ('n02104029', 'kuvasz', np.float32(0.13956322)),
  ('n02099601', 'golden_retriever', np.float32(0.05594526)),
  ('n02111500', 'Great_Pyrenees', np.float32(0.048894893)),
  ('n02100735', 'English_setter', np.float32(0.0021178788))]]

In [18]:
incep_res_prep_cat = incep_res_v2.preprocess_input(np.array(cat_png))
predictions = inception_resnet.predict(incep_res_prep_cat[np.newaxis,:])
incep_res_v2.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


[[('n02123045', 'tabby', np.float32(0.4249481)),
  ('n02124075', 'Egyptian_cat', np.float32(0.25831026)),
  ('n02123159', 'tiger_cat', np.float32(0.1279524)),
  ('n02127052', 'lynx', np.float32(0.003448608)),
  ('n04525038', 'velvet', np.float32(0.0024461092))]]